In [ ]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from tools.basic import load_usps, show_image,draw_pred
from sklearn.model_selection import train_test_split

from activation import *
from loss import *
from linear import * 
from encapsulage import *
from utils import *

from keras.datasets import mnist

def onehot(x): # single digit
    out = [0] * 10
    out[x] = 1
    return out
